In [ ]:
import cv2
from ultralytics import YOLO
import random
import tkinter as tk
from tkinter import filedialog

def process_video_with_tracking(model, input_video_path, show_video=True, save_video=False, output_video_path="output_video.mp4"):
    # Открываем видеофайл
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        raise Exception("Error: Could not open video file.")

    # Получаем частоту кадров и размеры кадра
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Определяем запись выходного видео, если требуется
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Запуск трекинга
        results = model.track(frame, iou=0.4, conf=0.5, persist=True, imgsz=608, verbose=False, tracker="botsort.yaml")

        # Если есть треки
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)

            for box, id_val in zip(boxes, ids):
                # Генерируем цвет на основе id
                random.seed(int(id_val))
                color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                
                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color, 2)
                cv2.putText(
                    frame,
                    f"Id {id_val}",
                    (box[0], box[1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 255),
                    2,
                )
        
        if save_video:
            out.write(frame)

        if show_video:
            # Можно изменить масштаб, если нужно
            # frame = cv2.resize(frame, (0, 0), fx=0.75, fy=0.75)
            cv2.imshow("frame", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    if save_video:
        out.release()
    cv2.destroyAllWindows()

def main():
    # Загрузка модели
    model = YOLO('runs/detect/train9/weights/best.pt')
    model.fuse()

    # Создаем главное окно Tkinter (не будем его показывать, оно нужно только для вызова диалога)
    root = tk.Tk()
    root.withdraw()

    # Диалог выбора файла
    input_video_path = filedialog.askopenfilename(
        title="Выберите видеофайл",
        filetypes=[("Video Files", "*.mp4;*.avi;*.mkv;*.mov;*.flv;*.wmv")]
    )

    if input_video_path:
        # Запускаем обработку выбранного видео
        process_video_with_tracking(model, input_video_path, show_video=True, save_video=False, output_video_path="output_video.mp4")
    else:
        print("Видео не было выбрано.")

if __name__ == "__main__":
    main()


YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
